# 测试集图像分类预测结果

调用自己训练的水果图像分类模型，对测试集所有图像进行图像分类预测，并将结果记录在表格中。

同济子豪兄：https://space.bilibili.com/1900783

[代码运行云GPU环境](https://featurize.cn/?s=d7ce99f842414bfcaea5662a97581bd1)：GPU RTX 3060、CUDA v11.2

2022-8-16

## 进入 mmclassification 目录

In [1]:
import os
os.chdir('mmclassification')

## 导入工具包

In [20]:
import matplotlib.pyplot as plt
%matplotlib inline

import numpy as np
import pandas as pd

from tqdm import tqdm

import torch
import mmcv
from mmcls.apis import init_model
from mmcls.datasets.pipelines import Compose

# 有 GPU 就用 GPU，没有就用 CPU
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print('device', device)

device cuda:0


## 将训练得到的模型在测试集上预测，获得图像分类评估结果

In [3]:
!python tools/test.py \
        configs/mobilenet_v2/mobilenet_v2_1x_fruit30.py \
        work_dirs/mobilenet_v2_1x_fruit30/latest.pth \
        --metrics accuracy precision recall f1_score support \
        --metric-options topk=1


/home/featurize/work/MMClassification教程/mmclassification/mmcls/utils/setup_env.py:33: UserWarning: Setting OMP_NUM_THREADS environment variable for each process to be 1 in default, to avoid your system being overloaded, please further tune the variable for optimal performance in your application as needed.
  f'Setting OMP_NUM_THREADS environment variable for each process '
/home/featurize/work/MMClassification教程/mmclassification/mmcls/utils/setup_env.py:43: UserWarning: Setting MKL_NUM_THREADS environment variable for each process to be 1 in default, to avoid your system being overloaded, please further tune the variable for optimal performance in your application as needed.
  f'Setting MKL_NUM_THREADS environment variable for each process '
load checkpoint from local path: work_dirs/mobilenet_v2_1x_fruit30/latest.pth
[>>>>>>>>>>>>>>>>>>>>>>>>>>>] 1078/1078, 147.4 task/s, elapsed: 7s, ETA:     0s
accuracy : 86.55

support : 1078.0

precision : 87.37

recall : 86.58

f1_score : 86.45


## 将训练得到的模型在测试集上预测，获得所有测试集数据的预测结果

### 方法一：MMClassification自带脚本

运行测试集预测脚本

In [4]:
!python tools/test.py \
        configs/mobilenet_v2/mobilenet_v2_1x_fruit30.py \
        work_dirs/mobilenet_v2_1x_fruit30/latest.pth \
        --out work_dirs/mobilenet_v2_1x_fruit30/testset_result.json

/home/featurize/work/MMClassification教程/mmclassification/mmcls/utils/setup_env.py:33: UserWarning: Setting OMP_NUM_THREADS environment variable for each process to be 1 in default, to avoid your system being overloaded, please further tune the variable for optimal performance in your application as needed.
  f'Setting OMP_NUM_THREADS environment variable for each process '
/home/featurize/work/MMClassification教程/mmclassification/mmcls/utils/setup_env.py:43: UserWarning: Setting MKL_NUM_THREADS environment variable for each process to be 1 in default, to avoid your system being overloaded, please further tune the variable for optimal performance in your application as needed.
  f'Setting MKL_NUM_THREADS environment variable for each process '
load checkpoint from local path: work_dirs/mobilenet_v2_1x_fruit30/latest.pth
[>>>>>>>>>>>>>>>>>>>>>>>>>>>] 1078/1078, 238.8 task/s, elapsed: 5s, ETA:     0s
dumping results to work_dirs/mobilenet_v2_1x_fruit30/testset_result.json


解析生成的json文件

In [3]:
import json
with open('work_dirs/mobilenet_v2_1x_fruit30/testset_result.json', 'r') as f:
    data = json.load(f)

In [4]:
# 载入类别名称 和 ID索引号 的映射字典
idx_to_labels = np.load('idx_to_labels.npy', allow_pickle=True).item()
# 获得类别名称
classes = list(idx_to_labels.values())
print(classes)

['哈密瓜', '圣女果', '山竹', '杨梅', '柚子', '柠檬', '桂圆', '梨', '椰子', '榴莲', '火龙果', '猕猴桃', '石榴', '砂糖橘', '胡萝卜', '脐橙', '芒果', '苦瓜', '苹果-红', '苹果-青', '草莓', '荔枝', '菠萝', '葡萄-白', '葡萄-红', '西瓜', '西红柿', '车厘子', '香蕉', '黄瓜']


In [5]:
df = pd.DataFrame()
for i in tqdm(range(len(data['pred_label']))):
    pred_dict = {}
    pred_dict['top-1-预测ID'] = data['pred_label'][i]
    pred_dict['top-1-预测名称'] = data['pred_class'][i]
    pred_dict['top-1-预测置信度'] = data['pred_score'][i]
    for class_id, pred_conf in enumerate(data['class_scores'][i]):
        pred_dict['{}-预测置信度'.format(idx_to_labels[class_id])] = pred_conf
    df = df.append(pred_dict, ignore_index=True)

100%|██████████| 1078/1078 [00:03<00:00, 335.94it/s]


In [6]:
df

,top-1-预测ID,top-1-预测名称,top-1-预测置信度,哈密瓜-预测置信度,圣女果-预测置信度,山竹-预测置信度,杨梅-预测置信度,柚子-预测置信度,柠檬-预测置信度,桂圆-预测置信度,...,草莓-预测置信度,荔枝-预测置信度,菠萝-预测置信度,葡萄-白-预测置信度,葡萄-红-预测置信度,西瓜-预测置信度,西红柿-预测置信度,车厘子-预测置信度,香蕉-预测置信度,黄瓜-预测置信度
0,0.0,哈密瓜,0.930069,9.300690e-01,3.509528e-04,2.360755e-05,8.278000e-06,3.848600e-03,9.749869e-03,1.524892e-05,...,1.266642e-05,7.372266e-06,4.068981e-06,5.011826e-05,2.464031e-06,1.095024e-04,1.381457e-03,4.367969e-06,2.431113e-04,2.283575e-05
1,8.0,椰子,0.869607,1.188348e-02,5.504910e-10,5.050380e-05,9.045514e-09,4.185473e-03,7.836395e-02,7.705273e-04,...,3.723392e-08,6.494256e-06,7.291856e-08,1.005571e-05,1.473000e-07,1.584690e-06,4.856365e-09,9.075064e-09,1.383865e-03,1.034700e-04
2,0.0,哈密瓜,0.447093,4.470935e-01,1.980640e-02,6.066528e-04,2.152952e-06,6.084440e-06,4.471582e-05,5.114610e-06,...,6.709216e-07,1.147448e-04,2.847370e-07,3.770713e-01,3.188694e-04,4.896535e-06,1.011590e-02,1.219882e-04,2.352822e-03,6.856504e-06
3,0.0,哈密瓜,0.999706,9.997059e-01,5.505493e-09,4.856195e-11,2.115697e-10,1.197204e-06,2.396194e-06,8.884907e-12,...,4.173593e-11,1.905787e-11,2.056154e-09,2.506672e-08,6.380712e-13,5.314835e-10,1.560283e-08,2.535498e-13,1.438733e-08,7.758549e-11
4,11.0,猕猴桃,0.407433,8.737191e-02,1.485267e-03,1.248030e-03,3.252485e-03,3.896382e-03,1.373343e-02,1.945081e-04,...,2.646697e-03,7.512841e-04,3.411181e-03,3.069140e-02,2.565478e-04,5.381129e-03,2.984692e-03,5.515811e-05,4.618626e-03,4.904474e-02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1073,29.0,黄瓜,0.974879,5.446826e-05,8.228727e-07,9.728999e-07,1.416050e-06,9.286327e-07,2.118429e-02,1.656441e-08,...,3.990991e-05,1.176539e-07,2.782325e-08,1.230811e-06,2.918328e-08,2.887484e-03,2.105287e-06,1.209993e-08,7.501955e-08,9.748793e-01
1074,29.0,黄瓜,0.999928,2.306059e-07,1.982897e-09,1.424076e-09,2.170595e-09,2.019479e-11,1.607080e-07,2.747951e-11,...,3.719310e-08,3.567485e-11,1.894585e-09,1.025917e-10,1.716963e-11,2.115243e-06,3.664796e-09,4.726995e-12,6.748072e-11,9.999282e-01
1075,29.0,黄瓜,0.966487,2.624635e-03,1.047371e-04,8.834132e-05,3.298897e-04,6.725066e-06,1.269468e-03,5.734710e-06,...,1.617144e-04,8.034226e-06,9.015609e-06,1.558806e-04,1.469086e-05,2.848643e-03,8.720866e-05,2.726350e-05,2.963160e-05,9.664874e-01
1076,29.0,黄瓜,0.992875,1.160476e-04,3.889816e-07,4.367774e-07,1.179589e-06,5.478886e-07,2.732908e-03,1.592856e-08,...,4.572861e-06,1.804956e-08,4.422348e-06,2.464426e-06,1.100523e-08,1.699554e-03,2.725396e-07,7.655003e-10,4.550510e-07,9.928753e-01


## 方法二：遍历测试集每张图像，手动预测图像分类结果

### A表-测试集文件名及标注类别

In [47]:
img_full_path_list = []
img_label_list = []
for fruit in os.listdir('fruit30_split/val'):
    for img_path in os.listdir(os.path.join('fruit30_split/val', fruit)):
        img_full_path = os.path.join('fruit30_split/val', fruit, img_path)
        img_full_path_list.append(img_full_path)
        img_label_list.append(fruit)
        
df_A = pd.DataFrame()
df_A['图像路径'] = img_full_path_list
df_A['标注类别名称'] = img_label_list

载入类别和ID的映射信息

In [48]:
idx_to_labels = np.load('idx_to_labels.npy', allow_pickle=True).item()
labels_to_idx = np.load('labels_to_idx.npy', allow_pickle=True).item()
df_A['标注类别ID'] = df_A['标注类别名称'].apply(lambda x:labels_to_idx[x])

In [49]:
df_A

,图像路径,标注类别名称,标注类别ID
0,fruit30_split/val/苦瓜/161.jpg,苦瓜,17
1,fruit30_split/val/苦瓜/158.jpg,苦瓜,17
2,fruit30_split/val/苦瓜/148.jpg,苦瓜,17
3,fruit30_split/val/苦瓜/183.jpg,苦瓜,17
4,fruit30_split/val/苦瓜/41.jpeg,苦瓜,17
...,...,...,...
1073,fruit30_split/val/火龙果/21.jpg,火龙果,10
1074,fruit30_split/val/火龙果/64.jpg,火龙果,10
1075,fruit30_split/val/火龙果/118.jpg,火龙果,10
1076,fruit30_split/val/火龙果/106.jpg,火龙果,10


### B表-测试集每张图像的图像分类预测结果

载入训练好的水果图像分类模型

In [50]:
# 训练得到的 mobilenet v2 新模型
config_file = 'configs/mobilenet_v2/mobilenet_v2_1x_fruit30.py'
checkpoint_file = 'work_dirs/mobilenet_v2_1x_fruit30/latest.pth'
# checkpoint_file = 'https://zihao-openmmlab.obs.myhuaweicloud.com/20220716-mmclassification/checkpoints/fruit30_mmcls/latest.pth'

# 通过 config 配置文件 和 checkpoint 权重文件 构建模型
model = init_model(config_file, checkpoint_file, device=device)

load checkpoint from local path: work_dirs/mobilenet_v2_1x_fruit30/latest.pth


In [51]:
cfg = model.cfg
# 去掉预处理的第一步，LoadImageFromFile
# 直接输入 img_array 而非 文件路径
cfg.data.test.pipeline.pop(0)
test_pipeline = Compose(cfg.data.test.pipeline)

In [54]:
df_B = pd.DataFrame()
for idx, row in tqdm(df_A.iterrows()):
    img_path = row['图像路径']
    img_array = mmcv.imread(img_path)

    # 预处理
    data = test_pipeline({'img': img_array})
    img = data['img'].unsqueeze(0).to(device)

    # 前向预测
    pred_softmax = model(img=img, return_loss=False, post_process=False)    
    
    ## 解析 top-n 预测类别和置信度
    top_n = torch.topk(pred_softmax, n)
    pred_ids = top_n[1].cpu().detach().numpy().squeeze() # 解析出类别
    confs = top_n[0].cpu().detach().numpy().squeeze() # 解析出置信度

    pred_dict = {}

    top_n = torch.topk(pred_softmax, n) # 取置信度最大的 n 个结果
    pred_ids = top_n[1].cpu().detach().numpy().squeeze() # 解析出类别
    
    # top-n 预测结果
    for i in range(1, n+1):
        pred_dict['top-{}-预测ID'.format(i)] = pred_ids[i-1]
        pred_dict['top-{}-预测名称'.format(i)] = idx_to_labels[pred_ids[i-1]]
    pred_dict['top-n预测正确'] = row['标注类别ID'] in pred_ids
    # 每个类别的预测置信度
    for idx, each in enumerate(classes):
        pred_dict['{}-预测置信度'.format(each)] = pred_softmax[0][idx].cpu().detach().numpy()
        
    df_B = df_B.append(pred_dict, ignore_index=True)

757it [00:14, 52.01it/s]Exception ignored in: <function _TemporaryFileCloser.__del__ at 0x7f37a510cef0>
Traceback (most recent call last):
  File "/environment/miniconda3/lib/python3.7/tempfile.py", line 448, in __del__
    self.close()
  File "/environment/miniconda3/lib/python3.7/tempfile.py", line 444, in close
    unlink(self.name)
FileNotFoundError: [Errno 2] No such file or directory: '/tmp/tmpf1zsq6za/tmp3jyl2agt.py'
1078it [00:21, 50.54it/s]


In [55]:
df_B

,top-1-预测ID,top-1-预测名称,top-2-预测ID,top-2-预测名称,top-3-预测ID,top-3-预测名称,top-n预测正确,哈密瓜-预测置信度,圣女果-预测置信度,山竹-预测置信度,...,草莓-预测置信度,荔枝-预测置信度,菠萝-预测置信度,葡萄-白-预测置信度,葡萄-红-预测置信度,西瓜-预测置信度,西红柿-预测置信度,车厘子-预测置信度,香蕉-预测置信度,黄瓜-预测置信度
0,17.0,苦瓜,14.0,胡萝卜,23.0,葡萄-白,1.0,1.5545739e-06,0.0013528649,1.7489974e-07,...,0.0015968541,1.3005163e-06,4.5650592e-07,0.0019987298,0.00049445196,4.040559e-05,0.00019853271,1.2931324e-07,4.4504642e-07,8.3786625e-05
1,17.0,苦瓜,23.0,葡萄-白,29.0,黄瓜,1.0,2.840711e-06,3.6908703e-05,8.258957e-09,...,1.9563948e-05,1.1485563e-07,1.9316593e-07,0.010118407,1.2882359e-05,3.0793074e-06,3.6499634e-06,1.6727906e-08,7.8847137e-07,0.00022895585
2,17.0,苦瓜,14.0,胡萝卜,29.0,黄瓜,1.0,2.5707911e-06,2.872274e-06,1.4406423e-10,...,2.0226682e-05,9.969936e-09,1.9336379e-07,1.0974867e-05,2.0791292e-07,3.5132973e-05,2.7435215e-07,5.043417e-10,4.636924e-07,0.0002881152
3,17.0,苦瓜,23.0,葡萄-白,14.0,胡萝卜,1.0,5.203559e-06,5.3404605e-05,3.957494e-09,...,9.399098e-05,1.2436384e-07,3.491051e-07,0.0010212788,1.8076169e-05,3.6820752e-06,5.000793e-06,1.7613177e-08,3.4024806e-06,2.3716328e-05
4,17.0,苦瓜,23.0,葡萄-白,20.0,草莓,1.0,1.0808951e-08,1.0108566e-07,8.95366e-12,...,3.7302175e-06,1.345433e-10,2.6345035e-08,0.0005139881,7.9986236e-08,1.1783909e-09,1.9784393e-08,6.9125435e-13,2.3080192e-08,5.676298e-07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1073,10.0,火龙果,24.0,葡萄-红,12.0,石榴,1.0,1.7312785e-11,1.1687246e-09,2.7255864e-10,...,5.014781e-08,6.310801e-13,4.4946755e-10,4.0481996e-09,1.832949e-07,1.1369534e-08,1.8542544e-11,5.2168946e-08,3.5493828e-09,1.0127483e-08
1074,10.0,火龙果,18.0,苹果-红,24.0,葡萄-红,1.0,7.8598123e-07,0.0030357195,0.00053314725,...,0.0031260003,2.9743182e-06,2.36687e-07,4.872642e-05,0.014315155,0.010274109,0.0003012552,0.00451395,9.976806e-06,0.00020246585
1075,10.0,火龙果,20.0,草莓,14.0,胡萝卜,1.0,2.0694078e-08,2.182299e-05,1.6188649e-07,...,0.0006327411,4.3393054e-08,8.585428e-08,3.2635077e-07,0.00032328954,3.9289218e-05,3.712329e-07,2.7670687e-05,8.165336e-07,8.033568e-06
1076,10.0,火龙果,27.0,车厘子,1.0,圣女果,1.0,4.8435344e-13,5.63452e-05,9.983658e-09,...,1.5860445e-05,7.137455e-11,5.825957e-14,2.5481997e-09,4.0906947e-05,2.9204753e-07,2.0333864e-08,0.0007718215,4.4085557e-10,1.077677e-07


## 拼接两张表格

In [56]:
df = pd.concat([df_A, df_B], axis=1)

In [57]:
df

,图像路径,标注类别名称,标注类别ID,top-1-预测ID,top-1-预测名称,top-2-预测ID,top-2-预测名称,top-3-预测ID,top-3-预测名称,top-n预测正确,...,草莓-预测置信度,荔枝-预测置信度,菠萝-预测置信度,葡萄-白-预测置信度,葡萄-红-预测置信度,西瓜-预测置信度,西红柿-预测置信度,车厘子-预测置信度,香蕉-预测置信度,黄瓜-预测置信度
0,fruit30_split/val/苦瓜/161.jpg,苦瓜,17,17.0,苦瓜,14.0,胡萝卜,23.0,葡萄-白,1.0,...,0.0015968541,1.3005163e-06,4.5650592e-07,0.0019987298,0.00049445196,4.040559e-05,0.00019853271,1.2931324e-07,4.4504642e-07,8.3786625e-05
1,fruit30_split/val/苦瓜/158.jpg,苦瓜,17,17.0,苦瓜,23.0,葡萄-白,29.0,黄瓜,1.0,...,1.9563948e-05,1.1485563e-07,1.9316593e-07,0.010118407,1.2882359e-05,3.0793074e-06,3.6499634e-06,1.6727906e-08,7.8847137e-07,0.00022895585
2,fruit30_split/val/苦瓜/148.jpg,苦瓜,17,17.0,苦瓜,14.0,胡萝卜,29.0,黄瓜,1.0,...,2.0226682e-05,9.969936e-09,1.9336379e-07,1.0974867e-05,2.0791292e-07,3.5132973e-05,2.7435215e-07,5.043417e-10,4.636924e-07,0.0002881152
3,fruit30_split/val/苦瓜/183.jpg,苦瓜,17,17.0,苦瓜,23.0,葡萄-白,14.0,胡萝卜,1.0,...,9.399098e-05,1.2436384e-07,3.491051e-07,0.0010212788,1.8076169e-05,3.6820752e-06,5.000793e-06,1.7613177e-08,3.4024806e-06,2.3716328e-05
4,fruit30_split/val/苦瓜/41.jpeg,苦瓜,17,17.0,苦瓜,23.0,葡萄-白,20.0,草莓,1.0,...,3.7302175e-06,1.345433e-10,2.6345035e-08,0.0005139881,7.9986236e-08,1.1783909e-09,1.9784393e-08,6.9125435e-13,2.3080192e-08,5.676298e-07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1073,fruit30_split/val/火龙果/21.jpg,火龙果,10,10.0,火龙果,24.0,葡萄-红,12.0,石榴,1.0,...,5.014781e-08,6.310801e-13,4.4946755e-10,4.0481996e-09,1.832949e-07,1.1369534e-08,1.8542544e-11,5.2168946e-08,3.5493828e-09,1.0127483e-08
1074,fruit30_split/val/火龙果/64.jpg,火龙果,10,10.0,火龙果,18.0,苹果-红,24.0,葡萄-红,1.0,...,0.0031260003,2.9743182e-06,2.36687e-07,4.872642e-05,0.014315155,0.010274109,0.0003012552,0.00451395,9.976806e-06,0.00020246585
1075,fruit30_split/val/火龙果/118.jpg,火龙果,10,10.0,火龙果,20.0,草莓,14.0,胡萝卜,1.0,...,0.0006327411,4.3393054e-08,8.585428e-08,3.2635077e-07,0.00032328954,3.9289218e-05,3.712329e-07,2.7670687e-05,8.165336e-07,8.033568e-06
1076,fruit30_split/val/火龙果/106.jpg,火龙果,10,10.0,火龙果,27.0,车厘子,1.0,圣女果,1.0,...,1.5860445e-05,7.137455e-11,5.825957e-14,2.5481997e-09,4.0906947e-05,2.9204753e-07,2.0333864e-08,0.0007718215,4.4085557e-10,1.077677e-07


In [59]:
df.to_csv('work_dirs/mobilenet_v2_1x_fruit30/测试集预测结果.csv', index=False)